In [0]:


DECLARE queryStr STRING;

SET VAR queryStr = "delete from  "|| :Catalog  ||".v_"|| :tableName  ||"_wide  where day_id =  date_format(date_add(now(), "|| :days  ||"),'yyyyMMdd'); ";
EXECUTE IMMEDIATE queryStr;

SET VAR queryStr = "        
        insert into  "|| :Catalog  ||".v_"|| :tableName  ||"_wide  
        with conf_market (select cast(SUB_MARKET_ID AS INT) SUB_MARKET_ID,SUB_MARKET_NAME,REGION,Time_Offset from hive_metastore.default.vz_conf_market)
        select 
        case when B.Time_Offset=-5 then  convert_timezone('UTC','America/New_York',A.request_ts_utc) 
              when B.Time_Offset=-6 then  convert_timezone('UTC','America/Chicago',A.request_ts_utc) 
              when B.Time_Offset=-8 then  convert_timezone('UTC','America/Los_Angeles',A.request_ts_utc) 
          end request_ts_local,
        REGION,A.* from (
            select 
            case when LEN(ne_id)=5 then left(ne_id,2) else left(ne_id,3)  end SUB_MARKET_ID,            
            * from "|| :Catalog  ||"."|| :tableName  ||"  
            where day_id =  date_format(date_add(now(), "|| :days  ||"),'yyyyMMdd')
        ) A
        left join conf_market B on A.SUB_MARKET_ID=B.SUB_MARKET_ID ;
";
EXECUTE IMMEDIATE queryStr;
--select *  from  invt.cm_gold.v_cm_invt_fw_4g_wide  where day_id =  date_format(date_add(now(), -1),"yyyyMMdd"); 
